In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

2021-12-13 15:53:15.883555: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-13 15:53:15.905848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 15:53:15.906107: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2070 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2021-12-13 15:53:15.906223: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-12-13 15:53:15.907288: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-13 15:53:15.908290: I tensorflow/stream_executor/platform/de

In [2]:
import gym
from RL_Agent import ddpg_agent, dqn_agent, dpg_agent, a2c_agent_discrete_queue, ppo_agent_discrete, \
    ppo_agent_discrete_parallel, dpg_agent_continuous, a2c_agent_continuous_queue, ppo_agent_continuous,\
    ppo_agent_continuous_parallel, a2c_agent_continuous
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input
from RL_Agent.base.utils.networks import networks
from IL_Problem.base.utils.callbacks import load_expert_memories, Callbacks
from RL_Problem import rl_problem
from IL_Problem.bclone import BehaviorCloning
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np


2021-12-13 15:53:15.940104: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-12-13 15:53:15.944510: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3199980000 Hz
2021-12-13 15:53:15.944866: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561c6f5937e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-13 15:53:15.944879: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-12-13 15:53:15.945030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 15:53:15.945453: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeF

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
environment = "LunarLander-v2"
environment = gym.make(environment)

In [4]:
exp_path = "tutorials/tf_tutorials/expert_demonstrations/Expert_LunarLander.pkl"
net_architecture = networks.dpg_net(dense_layers=2,
                                           n_neurons=[256, 256],
                                           dense_activation=['relu', 'relu'])

expert = dpg_agent.Agent(learning_rate=5e-4,
                         batch_size=32,
                         net_architecture=net_architecture)

expert_problem = rl_problem.Problem(environment, expert)

callback = Callbacks()

# Comentar si ya se dispone de un fichero de experiencias como "Expert_LunarLander.pkl"
print("Comienzo entrenamiento de un experto")
expert_problem.solve(1000, render=False, max_step_epi=250, render_after=980, skip_states=3)
expert_problem.test(render=False, n_iter=400, callback=callback.remember_callback)

callback.save_memories(exp_path)

Comienzo entrenamiento de un experto


2021-12-13 15:53:17.536546: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


Episode:  1 Epochs:  60  Reward: -114.2 Smooth Reward: -114.2  Epsilon: 0.0000
Episode:  2 Epochs:  92  Reward: -197.2 Smooth Reward: -155.7  Epsilon: 0.0000
Episode:  3 Epochs:  88  Reward: -32.3 Smooth Reward: -114.6  Epsilon: 0.0000
Episode:  4 Epochs:  81  Reward: -178.6 Smooth Reward: -130.6  Epsilon: 0.0000
Episode:  5 Epochs:  78  Reward: -160.0 Smooth Reward: -136.5  Epsilon: 0.0000
Episode:  6 Epochs:  86  Reward: -0.8 Smooth Reward: -113.8  Epsilon: 0.0000
Episode:  7 Epochs:  79  Reward: -71.8 Smooth Reward: -107.8  Epsilon: 0.0000
Episode:  8 Epochs:  78  Reward: -137.9 Smooth Reward: -111.6  Epsilon: 0.0000
Episode:  9 Epochs:  110  Reward: -311.1 Smooth Reward: -133.8  Epsilon: 0.0000
Episode:  10 Epochs:  67  Reward: -116.2 Smooth Reward: -132.0  Epsilon: 0.0000
Episode:  11 Epochs:  76  Reward: -116.4 Smooth Reward: -132.2  Epsilon: 0.0000
Episode:  12 Epochs:  99  Reward: -282.8 Smooth Reward: -140.8  Epsilon: 0.0000
Episode:  13 Epochs:  136  Reward: -82.7 Smooth Rewa

KeyboardInterrupt: 

In [16]:
def lstm_custom_model(input_shape):
    actor_model = Sequential()
    actor_model.add(LSTM(16, input_shape=input_shape, activation='tanh'))
    actor_model.add(Dense(128, input_shape=input_shape, activation='relu'))
    actor_model.add(Dense(128, input_shape=input_shape, activation='relu'))
    actor_model.add(Dense(128, activation='relu'))
    return actor_model

In [17]:
net_architecture = networks.actor_critic_net_architecture(use_custom_network=True,
                                                        actor_custom_network=lstm_custom_model,
                                                        critic_custom_network=lstm_custom_model
                                                        )

In [18]:
exp_path = "tutorials/tf_tutorials/expert_demonstrations/Expert_LunarLander.pkl"
use_action = True
n_stack = 5
exp_memory = load_expert_memories(exp_path, load_action=use_action, n_stack=n_stack)

In [9]:
agent = a2c_agent_discrete_queue.Agent(actor_lr=1e-5,
                                       critic_lr=1e-5,
                                       batch_size=32,
                                       epsilon=0.0,
                                       epsilon_decay=0.9999,
                                       epsilon_min=0.15,
                                       n_step_return=15,
                                       net_architecture=net_architecture,
                                       n_stack=n_stack)

In [10]:
bc = BehaviorCloning(agent, state_size=(n_stack, environment.observation_space.shape[0]), n_actions=environment.action_space.n,
                    n_stack=n_stack)

In [11]:
# bc = BehaviorCloning(agent, state_size=(n_stack, environment.observation_space.shape[0]), n_actions=environment.action_space.shape[0],
#                     n_stack=n_stack, action_bounds=[-1., 1.])

In [12]:
states = np.array([x[0] for x in exp_memory])
actions = np.array([x[1] for x in exp_memory])

In [13]:
agent = bc.solve(states, actions, epochs=10, batch_size=128, shuffle=True, optimizer=Adam(learning_rate=1e-4),
                 loss=tf.keras.losses.MeanSquaredError(),
                 metrics=tf.keras.metrics.MeanAbsoluteError(),
                 verbose=2,
                 validation_split=0.15, one_hot_encode_actions=False)

ValueError: in user code:

    /home/serch/TFM/CAPOIRL-TF2/RL_Agent/base/utils/networks/agent_networks.py:1774 _bc_train_step  *
        loss = self.bc_loss_func(y, y_)
    /home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:143 __call__  **
        losses = self.call(y_true, y_pred)
    /home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:246 call
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    /home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:1198 mean_squared_error
        return K.mean(math_ops.squared_difference(y_pred, y_true), axis=-1)
    /home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/tensorflow/python/ops/gen_math_ops.py:10038 squared_difference
        "SquaredDifference", x=x, y=y, name=name)
    /home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:744 _apply_op

In [ ]:
problem = rl_problem.Problem(environment, agent)
problem.test(render=True, n_iter=2)

In [ ]:
problem.solve(100, render=False, skip_states=1, max_step_epi=500)

In [ ]:
problem.test(render=True, n_iter=10)